## Approach 1 for clustering similar test steps in natural language

* Text embedding technique: Word2Vec
* Text similarity: Word Mover's Distance (WMD)
* Clustering techniques: Hierarchical agglomerative clustering and K-means

In [148]:
# Import necessary libraries
import os
import gc
import pandas as pd
import numpy as np
import math
import statistics as st
import re
import string
import time
import matplotlib.pyplot as plt

# For word frequency
from collections import defaultdict

# ML libraries
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
import gensim
from gensim.models import Word2Vec, Phrases, KeyedVectors
import random
from ast import literal_eval

# NLP libraries
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer 

# To be used with hierarchical clustering
from joblib import Memory

# To save models
import pickle

In [ ]:
# Download wordnet data through nltk
nltk.download('wordnet')

### Read data

### Data preprocessing functions

In [337]:
# Function to compute number of unique words in df
def get_number_unique_words(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [338]:
# Function to compute number of unique words in df ('test case name' field)
def get_number_unique_words_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [339]:
# Function to get list of words that occur less than a certain number of times
def get_word_frequency(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for step in test_steps:
        for word in step:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [340]:
# Function to get list of words that occur less than a certain number of times ('test case name' field)
def get_word_frequency_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for name in test_names:
        for word in name:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [341]:
# Function to remove problematic/mispelled words from vocabulary
def remove_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    current_dir = os.getcwd()
    up_dir = os.path.sep.join(current_dir.split(os.path.sep)[:-1])
    problematic_words = open(up_dir + '/word2vec_vocab_problematic.txt', 'r')
    problematic_words_list = list()
    for word in problematic_words:
        problematic_words_list.append(word.lstrip().rstrip())
    
    for index, row in df.iterrows():
        step = row["Steps"]
        df.loc[index]["Steps"] = [elem for elem in step if not elem in problematic_words_list]
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after removing problematic words: ", number_unique_words)

In [342]:
# function to fix problematic/mispelled words from vocabulary
def fix_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    current_dir = os.getcwd()
    up_dir = os.path.sep.join(current_dir.split(os.path.sep)[:-1])
    problematic_words = open(up_dir + '/word2vec_vocab_to_fix.txt', 'r')
    problematic_words_dict = {}
    for line in problematic_words:
        full_line = line.split(':')
        try:
            problematic_words_dict[full_line[0]] = [x.replace('\n', '') for x in full_line[1].split(',')]
        except:
            problematic_words_dict[full_line[0]] = full_line[1].replace('\n', '')
    
    for index, row in df.iterrows():
        step = row["Steps"]
        modified_step = list()
        for word in step:
            if word in problematic_words_dict:
                modified_step.extend(problematic_words_dict[word])
            else:
                modified_step.append(word)
        df.loc[index]["Steps"] = modified_step 
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after fixing problematic words: ", number_unique_words)

In [343]:
def preprocess_clean_data(df):
    print("Cleaning test case name field...")
    print("Dataset size before preprocessing: " , df.shape)
    
    # preprocessing and clean test name
    
    # replace url and similar structures (e.g, paths) with the keyword 'URL'
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(r'http\S+', 'URL', x))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('\/[\w-]*', '', x))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(r'\{[^)]*\}', '', x))
    
    # lowercase the step descriptions
    df["Case_Name"] = df["Case_Name"].apply(lambda x: x.lower())
    
    # remove digits and words with digits
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('\w*\d\w*','', x))
    
    # remove punctuations
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    # remove extra spaces
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(' +',' ',x))

    # tokenization
    df["Case_Name"] = df["Case_Name"].apply(lambda x: TweetTokenizer().tokenize(x))
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words across all test names: ", number_unique_words)
    
    # stopword removal
    stop_words = set(stopwords.words('english'))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: [w for w in x if not w in stop_words])
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words in test names after stopword removal: ", number_unique_words)
    
    # lemmatization
    lemmatizer = WordNetLemmatizer() 
    df["Case_Name"] = df["Case_Name"].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
    
    # remove words that occur a certain number of times
    ten_times_occurrence_words = get_word_frequency_name(df)
    print("Number of words that occurred only once in test case names: ", len(ten_times_occurrence_words))
    
    # list of words to be removed
    for index, row in df.iterrows():
        current_test_name = row["Case_Name"]
        list_words_to_remove = list()
        for word in current_test_name:
            if word in ten_times_occurrence_words:
                list_words_to_remove.append(word)
        
        df.loc[index]["Case_Name"] = [elem for elem in current_test_name if not elem in list_words_to_remove]

#     # remove single letters present in the data
#     df["Name"] = df["Name"].apply(lambda x: [w for w in x if len(w.strip()) > 1])

    # remove instances with empty names
    df = df.loc[df["Case_Name"] != '']
    
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words in test names in the end: ", number_unique_words)
    print("Dataset size after preprocessing: " , df.shape)

### Read and preprocess files with test cases and build dataframe

In [344]:
# Get data directory and list of xlsx files
current_dir = os.getcwd()
up_dir = os.path.sep.join(current_dir.split(os.path.sep)[:-2])
data_dir = up_dir + "\\filtered_data\\"
xlsxfiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(data_dir)
             for name in files
             if name.endswith((".xlsx"))]

In [ ]:
# Declare pandas df to be populated
column_names = ["Type", "Key", "Case_Name", "Step_ID", "Steps"]
test_steps_df = pd.DataFrame(columns = column_names)

# Index to add data to the df
index_to_add = 0

print("Reading input data...")   
for test_file in xlsxfiles:
    # load data and iterate through it
    test_data_df = pd.read_excel(test_file)
    for index, row in test_data_df.iterrows():
        current_type = row["Type"]
        current_key = row["Key"]
        current_name = row["Case_Name"]
        current_step_id = row["Step_ID"]
        current_steps = row["Steps"]
        test_steps_df.loc[index_to_add] = [current_type, current_key, current_name, current_step_id, current_steps]
        index_to_add += 1

print("Done!")
print("Shape of data => ", test_steps_df.shape)

In [ ]:
# Call preprocessing function
preprocess_clean_data(test_steps_df)

In [ ]:
# Preprocess steps here as for some reason its not processing properly within the function
# Replace urls with the keyword 'URL'
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(r'http\S+', 'URL', x))

# Remove structures similar to urls (e.g, paths) 
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('\/[\w-]*', '', x))
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(r'\{[^)]*\}', '', x))

# Lowercase the step descriptions
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: x.lower())

# Remove digits and words with digits
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('\w*\d\w*','', x))

# Remove punctuations
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

# Remove extra spaces
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(' +',' ',x))

# Tokenization
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: TweetTokenizer().tokenize(x))
number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words across all test steps: ", number_unique_words)

# Remove and fix mispelled words
remove_problematic_words(test_steps_df)
fix_problematic_words(test_steps_df)

# Stopword removal
stop_words = set(stopwords.words('english'))
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: [w for w in x if not w in stop_words])
number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words in test steps after stopword removal: ", number_unique_words)

# Lemmatization
lemmatizer = WordNetLemmatizer() 
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

# Remove words that occur only once in all test fields
ten_times_occurrence_words = get_word_frequency(test_steps_df)
print("Number of words that occurred only once in test steps: ", len(ten_times_occurrence_words))

for index, row in test_steps_df.iterrows():
    current_test_step = row["Steps"]
    list_words_to_remove = list()
    for word in current_test_step:
        if word in ten_times_occurrence_words:
            list_words_to_remove.append(word)

    test_steps_df.loc[index]["Steps"] = [elem for elem in current_test_step if not elem in list_words_to_remove]

number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words in test steps in the end: ", number_unique_words)
print("Dataset size after preprocessing: " , test_steps_df.shape)

In [ ]:
# Get only necessary fields to train word embedding models ('type', 'name', 'steps')
# Note: 'type' is always a str, while 'name' and 'steps' are always lists (even if they have a single element)
test_steps_training_list = list()
for index, row in test_steps_df_clean.iterrows():
    temp_list = list()
    if not pd.isnull(row["Type"]):
        temp_list.append(str(row["Type"]))
    
    if isinstance(row["Case_Name"], list):
        for elem in row["Case_Name"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Case_Name"], str):
            temp_list.append(row["Case_Name"])

    if isinstance(row["Steps"], list):
        for elem in row["Steps"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Steps"], str):
            temp_list.append(row["Steps"])
            
    # Build list of lists of tokens (words)
    test_steps_training_list.append(temp_list)

print("Length of list with training data: " , len(test_steps_training_list))

### Compute basic stats for the test steps

In [ ]:
# Total number of test steps
print("Total number of steps : ", len(test_steps_training_list))

# Average number of word per test step, name, and type together
total_number_words_steps = sum([len(steps) for steps in test_steps_training_list])
avg_words_per_step = total_number_words_steps/len(test_steps_training_list)
print("Average number of words per test step: ", avg_words_per_step)

# Most frequent words
word_freq = defaultdict(int)
for step in test_steps_training_list:
    for word in step:
        word_freq[str(word)] += 1
sorted(word_freq, key=word_freq.get, reverse=True)[:5]

### Train word embedding model - Word2Vec

In [ ]:
# Set parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 2   # Minimum word count (this should hold for all existing words now - one-occurring words were already removed)                       
num_workers = 4       # Number of threads to run in parallel (if necessary)
downsampling = 1e-3   # Downsample setting for frequent words
word2vec_model_list = []

for context in range(1,11,1):
    my_model = Word2Vec(workers=num_workers,
                       size=num_features,
                       min_count = min_word_count,
                       window = context, # Context window size
                       sample = downsampling)

    my_model.build_vocab(test_steps_training_list)
    total_examples = my_model.corpus_count
    print("Size of initial vocabulary: ", len(my_model.wv.vocab.keys()))

    # Load pre-trained model (set your custom path to where the model is located)
    pre_trained_model_path = "GoogleNews-vectors-negative300-002.bin"
    pre_trained_model = KeyedVectors.load_word2vec_format(pre_trained_model_path, binary=True)

    all_words = list(my_model.wv.vocab.keys())
    for each_word in all_words:
        my_model.wv[each_word] = np.zeros(300)

    # Update vocabulary with our corpus (must set 'lockf' to 1.0 to update model vocabulary)
    my_model.build_vocab([list(pre_trained_model.vocab.keys())], update=True)
    my_model.intersect_word2vec_format(pre_trained_model_path, binary=True, lockf=1.0)

    # Get mean and sd of initialized word vectors
    word_vector_median_list = list()
    count_initiliazed = 0
    for each_word in all_words:
        # if word vector was initialized
        if any(my_model.wv[each_word] != 0):
            word_vector_median_list.append(np.median(my_model.wv[each_word]))
            count_initiliazed += 1
    print("Number of words already initialized : " , count_initiliazed)

    # Define mean and sd for the normal distributions of medians        
    mu = np.mean(word_vector_median_list)
    sigma = np.std(word_vector_median_list)

    # Initialize the remaining word vectors (not present in the pre-trained model)
    count_manual_initiliazed = 0
    for each_word in all_words:
        # if word vector was NOT initialized
        if all(my_model.wv[each_word] == 0):
            # initialize this word vector
            new_word_vector = np.random.normal(mu, sigma, 300)
            my_model.wv[each_word] = new_word_vector
            count_manual_initiliazed += 1
    print("Number of words manually initialized : " , count_manual_initiliazed)

    my_model.train(test_steps_training_list, total_examples=total_examples, epochs=15)
    
    # Save the model (uncomment to save)
    path_save_model = 'word2vec_window_size_experiment/context_' + str(context) + '_my_model.model'
    my_model.save(path_save_model)
    word2vec_model_list.append(my_model)

In [200]:
# Define which model to use
model_index = 0
my_model = word2vec_model_list[model_index]

### Compute distance (WMD) between a sample of pairs of test steps

In [ ]:
# Select sample to evaluate embeddings
np.random.seed(30)
df_indices = np.random.randint(len(test_steps_df), size=5000)
test_steps_df_sampled = test_steps_df.iloc[df_indices]
len(test_steps_df_sampled)

In [ ]:
# Build tuples with (step_id, step_text) - used to retrieve the step ID in the end (after the clustering) - and get only test steps for clustering
step_id_text_tuple_list = list()
test_steps_clustering_list = list()
for index, row in test_steps_df_sampled.iterrows():
    step_id = row["Step_ID"]
    step_text = row["Steps"]
    step_id_text_tuple_list.append((step_id,step_text))

    temp_list = list()
    if isinstance(row["Steps"], list):
        for elem in row["Steps"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Steps"], str):
            temp_list.append(row["Steps"])
        
    # build list of lists of tokens (words)
    test_steps_clustering_list.append(temp_list)
    
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

In [ ]:
# Remove empty steps
index = 0
steps_to_remove = list()
for step in test_steps_clustering_list:
    if len(step) == 0:
        steps_to_remove.append(index)
    index += 1

step_id_text_tuple_list = [step_id_text_tuple_list[index] for index in range(len(step_id_text_tuple_list)) if not index in steps_to_remove]
test_steps_clustering_list = [test_steps_clustering_list[index] for index in range(len(test_steps_clustering_list)) if not index in steps_to_remove]
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

In [218]:
# Assign the number of test steps to rows and cols and initialize distance matrix with zeros
rows = cols = len(test_steps_clustering_list)
dist_matrix = np.zeros((rows, cols))

In [ ]:
start = time.perf_counter()
print("Computing distances....")
for row in range(rows):
    print(row)
    computed_dist_list = []
    for col in range(row+1, cols):
        computed_dist = my_model.wv.wmdistance(test_steps_clustering_list[row], test_steps_clustering_list[col])
        # Upper bound to avoid having inf values
        if computed_dist > 1000:
            computed_dist = 1000
        computed_dist_list.append(computed_dist)
    dist_matrix[row,row+1:] = dist_matrix[row+1:,row] = computed_dist_list

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

In [ ]:
# Print matrix's shape and check if it's correct
dist_matrix.shape

### Load ground truth, execute hierarchical agglomerative clustering and K-means, and compute F-score

In [135]:
# Function to compute the average word vector of a sentence (test step)
def sent_avg_vector(sent, my_model):
    list_word_vectors = list()
    num_words = 0
    for word in sent:
        list_word_vectors.append(my_model[word])
        num_words += 1
    sum_vectors = sum(list_word_vectors)
    avg_vector = sum_vectors/num_words    
    return avg_vector

In [160]:
# Compute average word vector
avg_word_sentence_vectors = list()
for sentence in test_steps_clustering_list:
    if len(sentence) > 0:
        avg_word_sentence_vectors.append(sent_avg_vector(sentence, my_model))  
print("========================")
print("Number of test steps: " , len(avg_word_sentence_vectors))
print("Dimension of one vector (should be 300 for word2vec)" , len(avg_word_sentence_vectors[0]))

Number of test steps:  15644
Dimension of one vector (should be 300 for word2vec) 300


C:\Users\markos\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


#### Load ground truth of similar test steps (to compute F-score)

In [ ]:
current_dir = os.getcwd() 
up_dir = os.path.sep.join(current_dir.split(os.path.sep)[:-1])
print(up_dir)

In [106]:
# Read excel files with ground truth of similar test steps (cannot be released due to sensitive information)
ground_truth_dir = up_dir + '/sample_manual_ground_truth/clusters/'
sample_files = os.listdir(ground_truth_dir)

In [107]:
manual_clusters_dict = {}
for sample in sample_files:
    sample_df = pd.read_excel(ground_truth_dir + sample)
    for index, row in sample_df.iterrows():
        cluster_id = row['cluster_id']
        step_id = row['step_id']
        if step_id in manual_clusters_dict:
            existing_list = manual_clusters_dict[step_id]
            existing_list.append(cluster_id)
            manual_clusters_dict[step_id] = existing_list
        else:
            manual_clusters_dict[step_id] = [cluster_id]

In [ ]:
print("Number of test step samples which were manually clustered: ", len(manual_clusters_dict))

In [109]:
test_steps_to_evaluate_list = list(manual_clusters_dict.keys())

### HAC and K-means with search for optimal number of clusters

In [ ]:
# Use F-score to tune number of clusters and evaluation
f_score_aggl_list = list()
f_score_kmeans_list = list()

# start = time.time()
time_hac = 0
time_kmeans = 0

list_num_clusters = list(range(50, 6001, 50))

# Iterate from 50 to 15,000 clusters
for num_clusters in list_num_clusters:
    start_time_hac = time.time()
    start_time_kmeans = time.time()
    
    print("Running for: " + str(num_clusters) + " clusters")
    aggl_clustering_model = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='average', memory=Memory('/tmp/memory_cache'))
    aggl_clustering_model.fit(dist_matrix)
    labels = aggl_clustering_model.labels_
    
    end_time_hac = time.time()
    time_hac = time_hac + (end_time_hac-start_time_hac)
    
    # Compute f-score - declare and initialize variables to compute F-score
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    appr_clusters_dict = {}
    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        for ind in indices_label:
            appr_clusters_dict[int(step_id_text_tuple_list[ind][0])] = single_label
        
    # Iterate through list of test steps to evaluate
    for i in range(len(test_steps_to_evaluate_list)-1):
        for j in range(i+1, len(test_steps_to_evaluate_list)):
            step_id_1 = test_steps_to_evaluate_list[i]
            step_id_2 = test_steps_to_evaluate_list[j]

            try:
                temp = appr_clusters_dict[step_id_1]
                temp2 = appr_clusters_dict[step_id_2]
            except:
                continue
    
            # true positive case
            if (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                TP += 1

            # false positive case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                FP += 1

            # false negative case
            elif (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                FN += 1

            # true negative case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                TN += 1
    
    try:
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f_score = (2 * precision * recall) / (precision + recall)
        f_score_aggl_list.append(f_score)
    except:
        continue
    
    
    # Declare and initialize numpy ndarray for k means centroid initialization with shape: [n_clusters, n_features]
    init_centroid_kmeans = np.zeros((num_clusters,300))

    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        sent_vectors_list = [avg_word_sentence_vectors[ind] for ind in indices_label]

        # Compute average sentence vector between the selected sentences in 'sent_vectors_list'
        avg_sent_vectors = sum(sent_vectors_list)/len(sent_vectors_list)
        init_centroid_kmeans[single_label,:] = avg_sent_vectors    
    
    # Run K-Means with the computed centroids
    kmeans = KMeans(n_clusters=num_clusters, init=init_centroid_kmeans, max_iter=300)
    kmeans.fit(avg_word_sentence_vectors)
    labels = kmeans.labels_
    
    end_time_kmeans = time.time()
    time_kmeans = time_kmeans + (end_time_kmeans-start_time_kmeans)
    
    # Compute f-score - declare and initialize variables to compute F-score
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    appr_clusters_dict = {}
    for single_label in set(labels):
        indices_label = np.where(labels == single_label)[0].tolist()
        for ind in indices_label:
            appr_clusters_dict[int(step_id_text_tuple_list[ind][0])] = single_label
        
    # Tterate through list of steps to evaluate
    for i in range(len(test_steps_to_evaluate_list)-1):
        for j in range(i+1, len(test_steps_to_evaluate_list)):
            step_id_1 = test_steps_to_evaluate_list[i]
            step_id_2 = test_steps_to_evaluate_list[j]
            
            try:
                temp = appr_clusters_dict[step_id_1]
                temp2 = appr_clusters_dict[step_id_2]
            except:
                continue
                
            # true positive case
            if (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                TP += 1

            # false positive case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
                FP += 1

            # false negative case
            elif (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                FN += 1

            # true negative case
            elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
                TN += 1
    try:
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f_score = (2 * precision * recall) / (precision + recall)
        f_score_kmeans_list.append(f_score) 
    except:
        continue
    
    # Delete unnecessary variables to free memory up
    del aggl_clustering_model
    del kmeans

### Analysis of hierarhical agglomerative clustering

In [ ]:
# Plot F-score against number of clusters
plt.plot(list(range(50, 6001, 50)), f_score_aggl_list)
plt.title('F-score - Agglomerative')
plt.xlabel('Number of clusters')
plt.ylabel('F-score')
plt.show()

In [ ]:
# Find max F-score and corresponding index
max_score = max(f_score_aggl_list)
max_index = f_score_aggl_list.index(max_score)
print("Best F-score: " , max_score)
print("Index of best F-score: ", max_index)

In [47]:
best_aggl_model = aggl_model_list[max_index]

In [ ]:
# Get number of clusters of the best model
best_aggl_model_num_clusters = best_aggl_model.n_clusters
best_aggl_model_labels = best_aggl_model.labels_
print("Number of clusters of the best agglomerative model: ", best_aggl_model_num_clusters)

### Analysis of K-means clustering

In [ ]:
# Plot F-score against number of clusters
plt.plot(list(range(50, 6001, 50)), f_score_kmeans_list)
plt.title('F-score - K-Means')
plt.xlabel('Number of clusters')
plt.ylabel('F-score')
plt.show()

In [ ]:
# Find max F-score and corresponding index
max_score = max(f_score_kmeans_list)
max_index = f_score_kmeans_list.index(max_score)
print("Best F-score: " , max_score)
print("Index of best F-score: ", max_index)

In [58]:
best_kmeans_model = kmeans_model_list[max_index]

In [ ]:
# Get number of clusters of the best model
best_kmeans_model_num_clusters = best_kmeans_model.n_clusters
best_kmeans_model_labels = best_kmeans_model.labels_
print("Number of clusters of the best k-means model: ", best_kmeans_model_num_clusters)
print("Number of non-empty clusters of the best k-means model: ", len(set(best_kmeans_model.labels_)))